# T5. fastNLP 与 paddle 或 jittor 的结合

&emsp; 1 &ensp; fastNLP 结合 paddle 训练模型
 
&emsp; &emsp; 1.1 &ensp; 

&emsp; &emsp; 1.2 &ensp; 

&emsp; 2 &ensp; fastNLP 结合 jittor 训练模型

&emsp; &emsp; 2.1 &ensp; 

&emsp; &emsp; 2.2 &ensp; 

&emsp; 3 &ensp; fastNLP 实现 paddle 与 pytorch 互转

&emsp; &emsp; 3.1 &ensp; 

&emsp; &emsp; 3.2 &ensp; 